# ***<u>librariess***

In [13]:
import numpy as np
from itertools import product
from collections import defaultdict
import random
import copy

# ***<u>Generate <u>Sudoku***

In [14]:
def generate_sudoku():

    """
        Function to generate a Sudoku grid

        
        Fill the diagonal boxes of the Sudoku grid
        Solve the Sudoku grid
        Remove elements from the grid to create a puzzle
    """
    sudoku = np.zeros((9, 9), dtype=int)
    fill_diagonal(sudoku) 
    solve_sudoku(sudoku)
    remove_elements(sudoku, num_elements=30) 
    return sudoku
    ############
    ##        ##
    ##  Code  ##
    ##        ##    
    ############



def fill_diagonal(grid):
    
    """
        Function to fill the diagonal boxes of the Sudoku grid
    """
    for i in range(0, 9, 3):
        fill_box(grid, i, i)


def fill_box(grid, rows, cols):

    """
        Function to fill a box with random values
    """
    nums = np.random.permutation(9) + 1
    ind=  0
    for row in range(rows, rows + 3):
        for col in range(cols,cols + 3):
            grid[row, col] = nums[ind]
            ind+=1
    ############
    ##        ##
    ##  Code  ##
    ##        ##
    ############

def is_safe(grid, row, col, num):

    """
        Function to check if it is safe to place a number in a particular position
    """

    return (
        not used_in_row(grid, row, num)
        and not used_in_column(grid, col, num)
        and not used_in_box(grid, row - row % 3, col - col % 3, num)
    )


# 
def used_in_row(grid, row, num):

    """
        Function to check if a number is used in a row
    """
    if num in grid[row, :]:
        return True
    ############
    ##        ##
    ##  Code  ##
    ##        ##
    ############


def used_in_column(grid, col, num):

    """
        Function to check if a number is used in a column
    """
    ############
    ##        ##
    ##  Code  ##
    ##        ##
    ############
    if num in grid[ :,col]:
        return True

def used_in_box(grid, box_start_row, box_start_col, num):

    """
        Function to check if a number is used in a 3x3 box
    """
    subgrid_row, subgrid_col=3 * (box_start_row//3),3 * (box_start_col//3)
    subgrid=grid[subgrid_row:subgrid_row + 3, subgrid_col:subgrid_col + 3]
    if num in subgrid:
        return True
    ############
    ##        ##
    ##  Code  ##
    ##        ##
    ############

def find_unassigned_location(grid):

    """
        Function to find an unassigned location in the grid
    """
    for row in range(9):
        for col in range(9):
            if grid[row, col] == 0 :
                return row, col
    return -1, -1
    ############
    ##        ##
    ##  Code  ##
    ##        ##
    ############
    

def remove_elements(grid, num_elements):
    # Ensure that we remove exactly num_elements from the grid
    count = 0
    while count < num_elements:
        row = random.randint(0, 8)
        col = random.randint(0, 8)
        if grid[row][col] != 0:
            grid[row][col] = 0
            count += 1
    return grid
    ############
    ##        ##
    ##  Code  ##
    ##        ##
    ############

# ***<u>BackTracking***

In [15]:
def solve_sudoku(grid, steps_counter=[0]):
    row, col = find_unassigned_location(grid)
    if (row, col) == (-1, -1):
        return True  # success!

    for num in range(1, 10):
        if is_safe(grid, row, col, num):
            grid[row, col] = num
            steps_counter[0] += 1  # Increment step counter
            if solve_sudoku(grid, steps_counter):
                return True
            grid[row, col] = 0  # Undo & try again

    return False  # Trigger backtracking

    ############
    ##        ##
    ##  Code  ##
    ##        ##
    ############
    

def display_grid(grid):

    """
        Function to display the Sudoku grid
    """

    for i in range(9):
        for j in range(9):
            print(grid[i, j], end=" ")
        print()

# ***<u>CSP***

In [16]:
def solve_sudoku_csp(grid,steps_counter=[0]):

    """
        Function to solve the Sudoku grid using Constraint Satisfaction Problem (CSP)
    """

    def create_domains(grid):

        """
            Function to create domains for each cell in the grid
        """
        domains={}
        for i in range(9):
            for j in range(9):
                if grid[i, j] == 0:
                    domains[(i,j)] = set(range(1, 10))
                else:
                    domains[(i, j)] = set([grid[i, j]])
        return domains
                
        ############
        ##        ##
        ##  Code  ##
        ##        ##
        ############

    def is_valid_assignment(i, j, val, assignment):

        """
            Function to check if assigning a value to a cell is valid

            Check if the value is already used in the same row
            Check if the value is already used in the same column
            Check if the value is already used in the same 3x3 box
        """
        for t in range(9):
            if assignment.get((i, t)) == val and t!= j:
                return False
            if assignment.get((t, j)) == val and t!= i:
                return False
        subgrid_row,subgrid_col = 3 * (i//3), 3 * (j//3)
        for x in range(subgrid_row, subgrid_row + 3):
            for y in range(subgrid_col, subgrid_col+3):
                if assignment.get((x, y)) == val and x!= i and y!= j:
                    return False
        return True
        ############
        ##        ##
        ##  Code  ##
        ##        ##
        ############
    def forward_check(i, j, val, domains):
        """
        Perform forward checking after assigning a value to a cell.
        Remove the assigned value from all related cells' domains.
        """
        for x in range(9):
            if x != j:
                domains[(i, x)].discard(val)
                if not domains[(i, x)]:
                    return False
            if x != i:
                domains[(x, j)].discard(val)
                if not domains[(x, j)]:
                    return False
        subgrid_row, subgrid_col = 3 * (i // 3), 3 * (j // 3)
        for x in range(subgrid_row, subgrid_row + 3):
            for y in range(subgrid_col, subgrid_col + 3):
                if x != i or y != j:
                    domains[(x, y)].discard(val)
                    if not domains[(x, y)]:
                        return False
        return True


    def find_unassigned_location(assignment):

        """
            Function to find an unassigned location in the grid
        """
        for i in range(9):
            for j in range(9):
                if (i, j) not in assignment or assignment[(i, j)] == 0:
                    return i, j
        return -1, -1
        ############
        ##        ##
        ##  Code  ##
        ##        ##
        ############

    def solve_csp(assignment, domains, steps_counter):
        steps_counter[0] += 1
        i, j = find_unassigned_location(assignment)
        if i == -1 and j == -1:
            return True  # Puzzle solved

        for val in domains[(i, j)]:
            if is_valid_assignment(i, j, val, assignment):
                assignment[(i, j)] = val
                new_domains = {key: values.copy() for key, values in domains.items()}
                if forward_check(i, j, val, new_domains):
                    if solve_csp(assignment, new_domains, steps_counter):
                        return True
                del assignment[(i, j)]

        return False  # Trigger backtrackin



    # Create initial domains for each cell in the grid
    domains = create_domains(grid)
    assignment = {(i, j): grid[i, j] for i in range(9) for j in range(9) if grid[i, j] != 0}
    if solve_csp(assignment, domains, steps_counter):
        # Reflect the assignment onto the grid
        for i in range(9):
            for j in range(9):
                grid[i][j] = assignment.get((i, j), 0)
        return True
    else:
        return False

# ***<u>Show <u>Result***

In [17]:
# Function to initialize the Sudoku grid
def initializing_grid():
    print("\nInitial Sudoku\n")
    sudoku_grid = generate_sudoku()
    for i in sudoku_grid:
        for j in i:
            print(j, end=' ')
        print("")
    print(end='\n\n\n\n\n')
    return sudoku_grid


# Function to solve the Sudoku grid using backtracking
def backtracking_answer(sudoku_grid):
    print("Backtracking Answer:\n\n")
    steps_counter = [0]  # Initialize step counter
    if solve_sudoku(sudoku_grid, steps_counter):
        print(f"Sudoku solved successfully ")
        display_grid(sudoku_grid)
    else:
        print("No solution exists for the given Sudoku.")
    print(end='\n\n\n\n\n')


# Function to solve the Sudoku grid using CSP
def csp_answer(sudoku_grid):
    print("CSP Answer:\n")
    steps_counter = [0]  # Initialize step counter
    if solve_sudoku_csp(sudoku_grid, steps_counter):
        print(f"Sudoku solved successfully using CSP in {steps_counter[0]} steps:")
        display_grid(sudoku_grid)
    else:
        print("No solution exists for the given Sudoku using CSP.")
    print(end='\n\n\n\n\n')

In [18]:
# Generate and display the initial Sudoku grid
initial_sudoku_grid = initializing_grid()

# Solve the Sudoku grid using backtracking
backtracking_sudoku = copy.deepcopy(initial_sudoku_grid)
backtracking_answer(backtracking_sudoku)

# Solve the Sudoku grid using CSP
csp_sudoku = copy.deepcopy(initial_sudoku_grid)
csp_answer(csp_sudoku)


Initial Sudoku

0 7 3 1 0 4 0 9 0 
9 0 2 5 3 6 0 0 8 
6 5 0 7 0 9 2 4 3 
0 6 8 2 4 0 0 7 0 
2 9 0 0 6 1 3 5 0 
1 3 4 0 5 7 0 6 0 
4 8 0 0 0 0 0 3 0 
0 2 6 0 1 5 4 0 0 
3 1 5 4 9 0 6 2 7 





Backtracking Answer:


Sudoku solved successfully 
8 7 3 1 2 4 5 9 6 
9 4 2 5 3 6 7 1 8 
6 5 1 7 8 9 2 4 3 
5 6 8 2 4 3 9 7 1 
2 9 7 8 6 1 3 5 4 
1 3 4 9 5 7 8 6 2 
4 8 9 6 7 2 1 3 5 
7 2 6 3 1 5 4 8 9 
3 1 5 4 9 8 6 2 7 





CSP Answer:

Sudoku solved successfully using CSP in 44 steps:
8 7 3 1 2 4 5 9 6 
9 4 2 5 3 6 7 1 8 
6 5 1 7 8 9 2 4 3 
5 6 8 2 4 3 9 7 1 
2 9 7 8 6 1 3 5 4 
1 3 4 9 5 7 8 6 2 
4 8 9 6 7 2 1 3 5 
7 2 6 3 1 5 4 8 9 
3 1 5 4 9 8 6 2 7 





